### Example) 마케팅 문구 검증 & 교정 Agent

In [ ]:
import os 
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

os.environ['OPENAI_API_KEY'] = ""

llm = ChatOpenAI(model_name="gpt-4o", 
                 temperature=0.7)

# style check prompt
style_check_prompt = ChatPromptTemplate.from_template(
    """
    다음 문장이 우리 Writing Style에 맞는지 검토해 주세요. 
    Writing Style: "간결하고 설득력 있는 문장을 사용하며, 친근한 톤을 유지합니다."

    사용자 문장: "{user_text}"

    결과:
    1. Writing Style과 얼마나 부합하는지 (0-100 점수)
    2. 어색한 표현이나 개선이 필요한 부분 설명
    """
)

# rewrite prompt
rewrite_prompt = ChatPromptTemplate.from_template(
    """
    다음 문장을 사람들이 더 잘 반응할 수 있도록 개선해 주세요. 
    Writing Style을 유지하면서 더 매력적인 문장을 만들어야 합니다.

    원래 문장: "{user_text}"

    개선된 문장:
    """
)

# LCEL 체인
style_check_chain = style_check_prompt | llm | StrOutputParser()
rewrite_chain = rewrite_prompt | llm | StrOutputParser()

full_chain = (
    RunnablePassthrough()
    .assign(style_feedback=style_check_chain)  # Writing Style 검수
    .assign(improved_text=lambda data: rewrite_chain.invoke({"user_text": data["user_text"]}))  # 문장 개선
)

# 실행 함수
def review_and_rewrite_text(user_text):
    result = full_chain.invoke({"user_text": user_text})  # LCEL 체인 실행

    print("Writing Style 검토 결과:")
    print(result["style_feedback"])

    print("개선된 문장:")
    print(result["improved_text"])
    
    return result

# 사용자 입력 받기
user_text = input("✍ 문장을 입력하세요: ")
result = review_and_rewrite_text(user_text)

## Logging & Example

In [7]:
import logging
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. 로깅 설정
logging.basicConfig(filename="llm_logs.txt", level=logging.INFO, format="%(asctime)s - %(message)s")

# 2. LLM 모델 설정
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.7)

# 3. 프롬프트 설정
#prompt = ChatPromptTemplate.from_template("문장을 고쳐 주세요: {user_text}")
prompt = PromptTemplate(
    template="문장을 고쳐 주세요: {user_text}",
    input_variables=["user_text"]
)

# 4. LLM 체인 구성
chain = prompt | llm | StrOutputParser()

# 5. LLM 호출 전 프롬프트 확인 함수
def debug_prompt(data, prompt):
    formatted_prompt = prompt.format(user_text=data["user_text"])
    print("\n LLM에 전달될 프롬프트:\n", formatted_prompt)
    return data  # 그대로 반환

# 6. LLM 응답을 로깅하는 함수
def log_response(data):
    response = data["response"]
    logging.info(f"LLM 응답: {response}")
    print("\n LLM 응답이 로그에 저장되었습니다.")
    return data  # 그대로 반환

# 7. 최종 결과를 커스텀 변환하는 함수 (예: 대문자로 변환)
def transform_output(data):
    transformed_text = data["response"].upper()  # 대문자로 변환
    print("\n 최종 변환된 결과:\n", transformed_text)
    return {"transformed_text": transformed_text}  # 변환된 결과 반환

# 8. LCEL 체인 구성
full_chain = (
    RunnableLambda(lambda data : debug_prompt(data, prompt))  # LLM 호출 전 프롬프트 확인
    .assign(response=chain)  # LLM 실행 후 응답 저장
    .assign(log=RunnableLambda(log_response))  # LLM 응답을 로깅
    .assign(final_result=RunnableLambda(transform_output))  # 최종 결과 변환
)

# 9. 실행 함수
def process_text(user_text):
    result = full_chain.invoke({"user_text": user_text})  # LCEL 체인 실행
    print("\n 최종 결과 (사용 가능):", result["final_result"])
    return result

# 10. 사용자 입력 받기
user_text = input("✍ 문장을 입력하세요: ")
result = process_text(user_text)

✍ 문장을 입력하세요:  복주머니 선물할 때마다 최대 1천원씩 보상금 받아요



 LLM에 전달될 프롬프트:
 문장을 고쳐 주세요: 복주머니 선물할 때마다 최대 1천원씩 보상금 받아요

 LLM 응답이 로그에 저장되었습니다.

 최종 변환된 결과:
 복주머니를 선물할 때마다 최대 1,000원의 보상금을 받아요.

 최종 결과 (사용 가능): {'transformed_text': '복주머니를 선물할 때마다 최대 1,000원의 보상금을 받아요.'}


In [8]:
result

{'user_text': '복주머니 선물할 때마다 최대 1천원씩 보상금 받아요',
 'response': '복주머니를 선물할 때마다 최대 1,000원의 보상금을 받아요.',
 'log': {'user_text': '복주머니 선물할 때마다 최대 1천원씩 보상금 받아요',
  'response': '복주머니를 선물할 때마다 최대 1,000원의 보상금을 받아요.'},
 'final_result': {'transformed_text': '복주머니를 선물할 때마다 최대 1,000원의 보상금을 받아요.'}}